<a href="https://colab.research.google.com/github/pragneshrana/WorkSample/blob/main/LLM/RAG_LANGCHAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir data
!wget -P data https://raw.githubusercontent.com/pixegami/langchain-rag-tutorial/main/data/books/alice_in_wonderland.md

--2024-01-22 09:30:22--  https://raw.githubusercontent.com/pixegami/langchain-rag-tutorial/main/data/books/alice_in_wonderland.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170002 (166K) [text/plain]
Saving to: ‘data/alice_in_wonderland.md’

alice_in_wonderland 100%[===================>] 166.02K  --.-KB/s    in 0.02s   

2024-01-22 09:30:23 (6.83 MB/s) - ‘data/alice_in_wonderland.md’ saved [170002/170002]



In [2]:
!pip install langchain
!pip install unstructured
!pip install sentence-transformers
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.7/228.7 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.9/457.9 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 57.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 40.4 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=d50c18306619bfb6abbd0ceceb82a79e16934712d58449458e03ae2

In [3]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
from langchain.evaluation import load_evaluator
import os
import shutil
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain.prompts import ChatPromptTemplate

DATA_PATH = "./data"
CHROMA_PATH = "chroma"



In [4]:
def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob="*.md")
    documents = loader.load()
    return documents

In [5]:
def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks



In [6]:
def save_to_chroma(chunks: list[Document]):
    # Clear out the database first.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    # create the open-source embedding function
    embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

    # Create a new DB from the documents.
    # db = Chroma.from_documents(
    #     chunks, embedding_function, persist_directory=CHROMA_PATH
    # )
    db = Chroma.from_documents(
        chunks, embedding_function)

    # print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")
    return db


In [7]:
def generate_data_store():
    documents = load_documents()
    chunks = split_text(documents)
    save_to_chroma(chunks)



In [8]:
import os

os.environ['OPENAI_API_KEY'] = 'dummy_key'

In [9]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
len(embedding_function.embed_query('Apple'))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

384

In [10]:
db = generate_data_store()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Split 1 documents into 801 chunks.
So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure of
making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a White Rabbit with pink eyes ran
close by her.
{'source': 'data/alice_in_wonderland.md', 'start_index': 1654}


In [11]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.evaluation import load_evaluator
from langchain.chat_models import LlamaEdgeChatService

In [12]:
 # Get embedding for a word.
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vector = embedding_function.embed_query("apple")
print(f"Vector for 'apple': {vector}")
print(f"Vector length: {len(vector)}")

Vector for 'apple': [-0.006138470955193043, 0.0310117918998003, 0.06479359418153763, 0.010941483080387115, 0.005267181899398565, -0.04747648909687996, 0.08120305091142654, 0.02898096665740013, 0.06676198542118073, 0.030300425365567207, 0.05746497958898544, -0.008623567409813404, 0.0013227633899077773, 0.0003991943085566163, -0.018843011930584908, -0.02579377591609955, -0.013042069040238857, -0.052624959498643875, -0.058292508125305176, -0.025899210944771767, -0.0333736427128315, 0.02456783689558506, -0.005226608831435442, 0.023005936294794083, 0.032860755920410156, 0.07502180337905884, 0.005801749881356955, -0.014958570711314678, -0.02875291369855404, -0.11855187267065048, -0.039321769028902054, -0.05138755589723587, 0.07661823183298111, 0.048403698951005936, -0.03025641292333603, -0.09143416583538055, 0.051182106137275696, -0.009649590589106083, -0.021510843187570572, -0.07177409529685974, -0.06322427839040756, -0.0176688339561224, 0.028081480413675308, 0.09004708379507065, 0.01941798

In [ ]:
# Prepare the DB.
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

query_text = 'How Does alice meet the mad hatter?'

# Search the DB.
results = db.similarity_search_with_relevance_scores(query_text, k=3)

if len(results) == 0 or results[0][1] < 0.7:
    print(f"Unable to find matching results.")

context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)
print(prompt)

model = ChatOpenAI()
response_text = model.predict(prompt)

sources = [doc.metadata.get("source", None) for doc, _score in results]
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)
